In [17]:
#All these packages need to be installed from pip
#For ML
import sklearn
import sklearn.feature_extraction.text
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups, make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer  #Feature extraction
from sklearn.naive_bayes import MultinomialNB #Our learner.
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, SelectFromModel
import pandas as pd

import nltk #For tokenizing and normalizing
import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
from matplotlib.colors import ListedColormap
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

%matplotlib inline

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import pandas
import requests
import json
import math

In [2]:
df = pandas.read_pickle('cmv_full_features.pkl')
df = df.sample(frac = .1)
#splitting data
data_train, data_test = train_test_split(df, test_size=0.3, random_state=123)
data_train['is_train'] = True
data_test['is_train'] = False

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#turn the training dataset into a tf-idf matrix
TFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_features = 10000, ngram_range=(1, 10),stop_words='english', norm='l2')
TFVects = TFVectorizer.fit_transform(data_train['com_text'])
TFVects.shape #(3836, 32241)
#print(TFVects.shape

(2685, 10000)

In [4]:
#Perform SVD on this matrix to reduce dimensionality

# SVD = TruncatedSVD(n_components=1000, random_state=123)
# reduced_data = SVD.fit_transform(TFVects)
# reduced_data.shape

(2685, 1000)

In [19]:
#combining tfidf features with liguistic features and clustering labels
tfdf = pd.DataFrame(TFVects.toarray())
features_train = pd.concat([tfdf, data_train.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
features_train.set_index('index', inplace = True)

In [22]:
#turn the test dataset into a tf-idf
TFVects_test = TFVectorizer.transform(data_test['com_text'])
#reduced_test = SVD.transform(TFVects_test)
tfdf_test = pd.DataFrame(TFVects_test.toarray())
features_test = pd.concat([tfdf_test, data_test.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
features_test.set_index('index', inplace = True)

In [7]:
X = features_train
y = data_train['com_delta_received'].as_matrix()

# Idea 1: try classification for all posts

## Logistic regression

In [ ]:
#combining tfidf features with liguistic features and clustering labels
# tfdf = pd.DataFrame(TFVects.toarray())
# features_train = pd.concat([tfdf, data_train.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
# features_train.set_index('index', inplace = True)


In [ ]:
#use scree plot to determine the number of dimensions
# n = TFVects.shape[0]
# fig = plt.figure(figsize=(12,5))
# ax1 = fig.add_subplot(121)
# eigen_vals = np.arange(n) + 1
# ax1.plot(eigen_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
# ax1.set_title('Scree Plot')
# ax1.set_xlabel('Principal Component')
# ax1.set_ylabel('Proportion of Explained Variance')

# ax2 = fig.add_subplot(122)
# eigen_vals = np.arange(10) + 1
# ax2.plot(eigen_vals, pca.explained_variance_ratio_[:10], 'ro-', linewidth=2)
# ax2.set_title('Scree Plot (First 20 Principal Components)')
# ax2.set_xlabel('Principal Component')
# ax2.set_ylabel('Proportion of Explained Variance')
# plt.show()

In [ ]:
#predicting with first ten
# X = reduced_data[:, :2]
# Y = np.array([int(label) for label in data_train['com_delta_received']]) #Transform our predictor variable. 
              
#fitting logistic regresion
# logistic = linear_model.LogisticRegression()
# logistic.fit(X, Y)
# print("This logistic model using top two components fits {} of our training set".format(logistic.score(X,Y)))


In [ ]:
# #turn the test dataset into a tf-idf
# TFVects_test = TFVectorizer.transform(data_test['com_text'])
# tfdf_test = pd.DataFrame(TFVects_test.toarray())
# features_test = pd.concat([tfdf_test, data_test.reset_index()[['index', 'com_upvotes', 'KL', 'JS', 'kmeans', 'com_avg_pt_depth']]], axis = 1, ignore_index = False)
# features_test.set_index('index', inplace = True)
# reduced_data_test = pca.transform(features_test)

In [ ]:
# X_test = reduced_data_test[:, :2]
# Y_test = np.array([int(label) for label in data_test['com_delta_received']])
# print("This logistic model using top ten components fits {} of our testing set".format(logistic.score(X_test, Y_test)))

In [ ]:
# #Evaluation stats
# print("Precision:", sklearn.metrics.precision_score(Y, logistic.predict(X), average = 'weighted')) #precision
# print("Recall:",sklearn.metrics.recall_score(Y, logistic.predict(X), average = 'weighted')) #recall
# print("F-measure:",sklearn.metrics.f1_score(Y, logistic.predict(X), average = 'weighted')) #F-1 measure

In [ ]:
# #ROC curve
# x, y, _ = sklearn.metrics.roc_curve(Y, logistic.predict_proba(X)[:,1])
# roc_auc = sklearn.metrics.auc(x,y)

# plt.figure()
# plt.plot(x,y, color = 'darkorange', lw = 2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic curve')
# plt.legend(loc="lower right")
# plt.show()

In [ ]:
#Problem: we lost track of the feature names after PCA

## Binary Naive Bayes

## Decision tree

In [ ]:
# tree = DecisionTreeClassifier(max_depth=4,random_state=0).fit(reduced_data, data_train['com_delta_received'])

In [ ]:
# labels = tree.predict(reduced_data_test)

In [ ]:
# mat = confusion_matrix(data_test['com_delta_received'], labels)
# seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#                 xticklabels=['delta_not_received', 'delta_received'], yticklabels=['delta_not_received', 'delta_received'])
# plt.xlabel('true label')
# plt.ylabel('predicted label');

In [ ]:
# print('Precision: {}'.format(sklearn.metrics.precision_score(data_test['com_delta_received'], labels, average = 'weighted'))) 
# print('Recall: {}'.format(sklearn.metrics.recall_score(data_test['com_delta_received'], labels, average = 'weighted'))) 
# print('F1 Score: {}'.format(sklearn.metrics.f1_score(data_test['com_delta_received'], labels, average = 'weighted'))) 

# labels = [1 if dr else 0 for dr in labels]
# probs = tree.predict_proba(reduced_data_test)
# print('AUC Score: {}'.format(sklearn.metrics.roc_auc_score(data_test['com_delta_received'], probs[:,1])))



## Neural Nets - multi-layer perceptron

In [ ]:
# #initialize the model
# mlp_clf = MLPClassifier()

# fit the model
# mlp_clf.fit(X, y)

# #mlp_labels = [mlp_clf.predict(v)[0] for v in features_test]

In [ ]:
# #Confusion matrix
# mat = confusion_matrix(data_test['com_delta_received'], mlp_labels)
# seaborn.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
#                 xticklabels=['delta_not_received', 'delta_received'], yticklabels=['delta_not_received', 'delta_received'])
# plt.xlabel('true label')
# plt.ylabel('predicted label')

In [ ]:
#data_test['com_delta_received']
#mlp_labels

In [ ]:
# #ROC curve
# x, y, _ = sklearn.metrics.roc_curve(data_test['com_delta_received'], mlp_labels)
# roc_auc = sklearn.metrics.auc(x,y)

# plt.figure()
# plt.plot(x,y, color = 'darkorange', lw = 2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic curve')
# plt.legend(loc="lower right")
# plt.show()

# Idea 2: try classification sparately for each "cluster"

# Feature extraction

Idea: We realized that none of the implementations of the machine learning algorithms actually provides us with the list of features that are most reliable to classify comments. Thus, we are using feature extraction tools from sklearn.

Oh, we've observed that the features are very stable across algorithms, we could interpret these features by finding them in comments and see if these comments receive delta or not

## feature extraction using SelectKBest

In [10]:
def extract_KBest(func, k):
    selector = SelectKBest(func, k=k).fit(X,y)
    feature_indices = selector.get_support(indices=True)
    
    selected_features = features_train.columns[feature_indices].get_values()
    scores = selector.scores_[feature_indices]
    if selector.pvalues_ != None:
        pvalues = selector.pvalues_[feature_indices]
    else:
        pvalues = None
    return (selected_features, scores, pvalues)

In [ ]:
#f_classif
selected_features_f, scores_f, pvalues_f = extract_KBest(sklearn.feature_selection.f_classif, 20)
selected_features_f

In [ ]:
tf_features_indices_f = selected_features_f[:-1].astype(int)
all_features_f  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_f].tolist()
all_features_f.append(selected_features_f[-1])
#all_features_f
d = {'selected_features_f': selected_features_f, 'scores_f': scores_f, 'pvalues_f': pvalues_f}
F_f_DF = pd.DataFrame(data = d).sort(columns='scores_f', axis=0, ascending=False).reset_index(drop = True)
F_f_DF

In [ ]:
#chi2 - we can't use this algo because some values of upvotes are negative


In [ ]:
#mutual_info_classif
selected_features_m, scores_m, pvalues_m = extract_KBest(mutual_info_classif, 20)
selected_features_m

In [ ]:
pvalues_m

In [ ]:
tf_features_indices_m = selected_features_m[:-1].astype(int)
all_features_m  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_m].tolist()
all_features_m.extend(selected_features_m[-1:])
#all_features_m
d = {'selected_features_m': selected_features_m, 'scores_m': scores_m}
F_m_DF = pd.DataFrame(data = d).sort(columns='scores_m', axis=0, ascending=False).reset_index(drop = True)
F_m_DF

In [ ]:
F_m_DF

## Feature extraction using Tree-based feature selection

In [ ]:
def extract_model(clf_fitted):
    feature_indices = SelectFromModel(clf_fitted, prefit=True).get_support(indices=True)
    try:
        score = clf_fitted.feature_importances_[feature_indices]
    except:
        score = None
    selected_features = features_train.columns[feature_indices].get_values()
    return (selected_features, score)

In [ ]:
#Decision tree
tree_clf = DecisionTreeClassifier(max_depth=10,random_state=0).fit(X, y)

In [ ]:
selected_features_t1, scores_t1 = extract_model(tree_clf)
selected_features_t1

In [ ]:
selected_features_t1.shape

In [ ]:
tf_features_indices_t1 = selected_features_t1[:-2].astype(int)
all_features_t1  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_t1].tolist()
all_features_t1.extend(selected_features_t1[-2:])
#all_features_t1

d = {'selected_features_t1': selected_features_t1, 'scores_t1': scores_t1}
F_t1_DF = pd.DataFrame(data = d).sort(columns='scores_t1', axis=0, ascending=False).reset_index(drop = True)
F_t1_DF.iloc[:20, :]

In [ ]:
#Extra decision tree
#fits a number of randomized decision trees (a.k.a. extra-trees) on various sub-samples of the dataset 
#and use averaging to improve the predictive accuracy and control over-fitting
tree2_clf = ExtraTreesClassifier(n_estimators=250,random_state=0).fit(X, y)

In [ ]:
selected_features_t2, scores_t2 = extract_model(tree2_clf)
#selected_features_t2[2700:]

In [ ]:
#selected_features_t2[2600:]

In [ ]:
tf_features_indices_t2 = selected_features_t2[:-5].astype(int)
all_features_t2  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_t2].tolist()
all_features_t2.extend(selected_features_t2[-5:])
#all_features_t1

d = {'selected_features_t2': selected_features_t2, 'scores_t2': scores_t2}
F_t2_DF = pd.DataFrame(data = d).sort(columns='scores_t2', axis=0, ascending=False).reset_index(drop = True)
F_t2_DF.iloc[:20, :]

In [ ]:
F_DF = pd.concat([F_f_DF, F_m_DF, F_t1_DF.iloc[:20, :], F_t2_DF.iloc[:20, :]], axis =1)
F_DF

In [ ]:
## Feature extraction using Neural nets - mlp

# log_clf = linear_model.LogisticRegression()
# log_clf.fit(X, y)
# selected_features_log, scores_log = extract_model(log_clf)
# selected_features_log.shape
# selected_features_log[4500:]
# tf_features_indices_log = selected_features_log[:-1].astype(int)
# all_features_log = np.array(TFVectorizer.get_feature_names())[tf_features_indices_log].tolist()
# all_features_log.extend(selected_features_log[-1:])
#all_features_t1

# d = {'all_features_log': all_features_log, 'scores_log': scores_log}
# F_log_DF = pd.DataFrame(data = d).sort(columns='scores_log', axis=0, ascending=False).reset_index(drop = True)
# F_log_DF.iloc[:20, :]
#log_decisionDF = pd.DataFrame(log_clf.decision_function(X))#.sort(columns = [0], ascending = False)
#log_clf.densify(）

In [ ]:
F_DF.to_pickle("extracted_features.pkl")

In [16]:
#ef = pandas.read_pickle('extracted_features.pkl')
#ef

,pvalues_f,scores_f,selected_features_f,scores_m,selected_features_m,scores_t1,selected_features_t1,scores_t2,selected_features_t2
0,0.000000e+00,2598.615876,com_upvotes,0.029173,com_upvotes,0.248368,com_upvotes,0.029779,com_upvotes
1,1.795970e-60,270.443076,2,0.006716,2,0.025082,4,0.002256,2
2,2.971789e-13,53.280571,JS,0.006125,4,0.022398,341,0.002035,4
3,3.674229e-13,52.862969,KL,0.005891,3,0.015484,616,0.001906,30
4,1.437484e-09,36.642989,com_avg_pt_depth,0.005411,8,0.015049,143,0.001873,28
5,7.767745e-06,20.002018,248,0.005364,0,0.014497,280,0.001866,8
6,1.100916e-05,19.335121,3,0.004977,5,0.014274,453,0.001843,5
7,4.849321e-05,16.511506,553,0.004642,13,0.013842,435,0.001824,7
8,5.171193e-05,16.389600,149,0.004641,6,0.013674,896,0.001761,19
9,1.249823e-04,14.720291,404,0.004118,com_avg_pt_depth,0.013339,329,0.001754,18


I still want to get the real words, let me try to do it from here

Basically, 
- <b>TFVects</b> is the huge sparse matrix that cannot be convert to a dense matrix
- <b>reduced_data</b> is the dense matrix that has 1000 ngram features
- <b>features_train</b> is the dataframe that has 1000 ngram features + others we obtained

The problem is that I know the indices of the ngrams in <b>features_train</b>, and I want to figure out what these indices correspond to in <b>TFVects</b>, the huge sparse matrix

In [11]:
#f_classif
# selected_features_f, scores_f, pvalues_f = extract_KBest(sklearn.feature_selection.f_classif, 20)
# selected_features_f

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


array([0, 14, 65, 81, 160, 191, 304, 319, 397, 417, 448, 523, 588, 695,
       705, 827, 951, 959, 980, 'com_upvotes'], dtype=object)

In [ ]:
# tf_features_indices_f = selected_features_f[:-1].astype(int)
#-> indices within the 1000 dense matrix

(1000, 21560)

In [ ]:
# all_features_f  = np.array(TFVectorizer.get_feature_names())[tf_features_indices_f].tolist()
# all_features_f.append(selected_features_f[-1])
# all_features_f

In [ ]:
# d = {'selected_features_f': selected_features_f, 'scores_f': scores_f, 'pvalues_f': pvalues_f}
# F_f_DF = pd.DataFrame(data = d).sort(columns='scores_f', axis=0, ascending=False).reset_index(drop = True)
# F_f_DF